In [95]:
!pip install keras_nlp

/bin/bash: /opt/anaconda-3-2020.02/envs/hskim/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 573 kB 10.7 MB/s            
     |████████████████████████████████| 924 kB 16.6 MB/s            
     |████████████████████████████████| 6.5 MB 16.9 MB/s            
     |████████████████████████████████| 130 kB 12.9 MB/s            
     |████████████████████████████████| 239 kB 16.4 MB/s            
     |████████████████████████████████| 776 kB 15.5 MB/s            
     |████████████████████████████████| 152 kB 15.8 MB/s            
     |████████████████████████████████| 4.8 MB 16.2 MB/s            
  Using cached Pygments-2.16.1-py3-none-any.whl (1.2 MB)
     |████████████████████████████████| 87 kB 11.1 MB/s            
     |████████████████████████████████| 85 kB 7.1 MB/s             
     |████████████████████████████████| 479.6 MB 29 kB/

In [100]:
!pip install tensorflow-text

/bin/bash: /opt/anaconda-3-2020.02/envs/hskim/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
#import sys
#sys.path.append('./.local/lib/python3.8/site-packages')
#sys.path.append('/usr/lib/python3/dist-packages')

#import matplotlib.pyplot as plt

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.preprocessing.text import Tokenizer, one_hot, text_to_word_sequence
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras_nlp.tokenizers import WordPieceTokenizer
from keras.models import model_from_json

BATCH_SIZE = 256
NUM_EPOCH = 100

GPU = 3
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]= f"{GPU}"  

2023-11-17 15:05:28.714313: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-17 15:05:33.430490: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


In [2]:
import csv
import pandas as pd

df_vocab = pd.read_csv(f'vocab/VOCABULARY.csv', sep='\t', on_bad_lines='error')
df_concept_rel = pd.read_csv(f'vocab/CONCEPT_RELATIONSHIP.csv', sep='\t', on_bad_lines='error')
df_concept = pd.read_csv(f'vocab/CONCEPT.csv', sep='\t')

/tmp/ipykernel_1364053/1335861446.py:6: DtypeWarning: Columns (5,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_concept = pd.read_csv(f'vocab/CONCEPT.csv', sep='\t')


In [3]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, droprate):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.droprate = droprate
        self.ff_dim = ff_dim
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential([layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(droprate)
        self.dropout2 = layers.Dropout(droprate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
        return {'embed_dim': self.embed_dim, 'num_heads': self.num_heads, 'ff_dim':ff_dim, 'droprate':self.droprate}

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.maxlen = maxlen
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

    def get_config(self):
        return {'maxlen': self.maxlen, 'vocab_size': self.vocab_size, 'embed_dim': self.embed_dim}

In [4]:
# CDC master
# https://view.officeapps.live.com/op/view.aspx?src=https%3A%2F%2Fwww.cdc.gov%2Fnhsn%2Fxls%2Ficd10-pcs-pcm-nhsn-opc.xlsx&wdOrigin=BROWSELINK
input_path = 'icd10pcs_mapping'
df1 = pd.read_csv(f'{input_path}/icd10-pcs-pcm-nhsn-opc.csv', dtype=str)
df1.rename(columns={'ICD-10-PCS Codes':'code', 'Procedure Code Descriptions':'name'}, inplace=True)
df1['code'] = df1['code'].str[:-1]   # remove empty space in the end part of code
df1['src'] = 'cdc'

# Centers for Medicare & Medicaid Services
# https://www.cms.gov/medicare/icd-10/2022-icd-10-pcs
df2 = pd.read_fwf(f'{input_path}/icd10pcs_order_2022.txt', widths=(6,7,3,60), header=None, dtype=str)
df2.rename(columns={1:'code',3:'name'}, inplace=True)
df2['src'] = 'cms'

# SNUH data
# system, operation, body part, approach
#df3 = pd.read_csv('icd10_mapping_with_final.csv', dtype=str)
df3 = pd.read_csv(f'{input_path}/icd10_mapping_hclee.csv', usecols=['opid', 'p', 'o', 'a', 'opname'], dtype=str)  # the label manually checked by hclee
#df3.loc[df3['opname_final'] == df3['opname'], 'opname_final'] = ''
# merge the name with opname and opname_final
df3['name'] = df3['opname']
#df3['name'] = None
#df3.loc[df3['opname'].notnull() & df3['opname_final'].notnull(), 'name'] = df3['opname'] + '\r' + df3['opname_final']
#df3.loc[df3['opname'].notnull() & df3['opname_final'].isnull(), 'name'] = df3['opname']
#df3.loc[df3['opname'].isnull() & df3['opname_final'].notnull(), 'name'] = df3['opname_final']
df3['src'] = 'snuh'

# MIMIC
df4 = pd.read_csv(f'{input_path}/d_icd_procedures.csv.gz', compression='gzip')
df4.rename(columns={'icd_code':'code','long_title':'name'}, inplace=True)
df4 = df4[df4['icd_version'] == 10]
df4['src'] = 'mimic'

# merge all
df = pd.concat([df1, df2, df3, df4])

# remove the row without op name
df.dropna(subset='name', inplace=True)

# split the part from the icd code for mimic, cdc, and cms
df.loc[df['code'].str.len() < 5, 'code'] = None
df.loc[df['code'].str[0] != '0', 'code'] = None
null_mask = df['p'].isnull()
df.loc[null_mask, 'p'] = df.loc[null_mask, 'code'].str[1] + df.loc[null_mask, 'code'].str[3]
null_mask = df['o'].isnull()
df.loc[null_mask, 'o'] = df.loc[null_mask, 'code'].str[2] #+ df.loc[null_mask, 'code'].str[2]
null_mask = df['a'].isnull()
df.loc[null_mask, 'a'] = df.loc[null_mask, 'code'].str[4] #+ df.loc[null_mask, 'code'].str[4]

In [91]:
len(df3), df3['p'].isna().sum(), df3['o'].isna().sum(), df3['a'].isna().sum()

(286215, 9999, 8045, 4991)

### eda - SNOMED- relationship

In [53]:
snomed_vocabs = df_concept[(df_concept['vocabulary_id']=='SNOMED') & (df_concept['domain_id']=='Procedure')]
snomed_vocabs

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
6903267,4001100,Resection of stomach fundus,Procedure,SNOMED,Procedure,S,10002003,20020131,20991231,NaN
6903439,4002094,Repair of retinal detachment with xenon arc ph...,Procedure,SNOMED,Procedure,S,10006000,20020131,20991231,NaN
6903573,40264452,Retired procedure,Procedure,SNOMED,Procedure,NaN,10009007,19700101,20020131,D
6903613,4000799,Cauterization of Bartholin's gland,Procedure,SNOMED,Procedure,S,1001000,20020131,20991231,NaN
6903617,42534812,Ultrasonography of calf of right lower leg,Procedure,SNOMED,Procedure,S,1001000087107,20180131,20991231,NaN
...,...,...,...,...,...,...,...,...,...,...
7957113,40648609,Fitting of prosthesis or prosthetic device of arm,Procedure,SNOMED,Procedure,NaN,9987003,20020131,20070131,D
7957156,4321416,Computed tomography oblique,Procedure,SNOMED,Procedure,S,9988008,20020131,20991231,NaN
7957419,4322524,Radiation therapy treatment planning service,Procedure,SNOMED,Procedure,S,9990009,20020131,20991231,NaN
7957800,4322532,Incision of diaphragm,Procedure,SNOMED,Procedure,S,9993006,20020131,20991231,NaN


In [20]:
icd10pcs_ids = df_concept.loc[df_concept['vocabulary_id']=='ICD10PCS', 'concept_id'].values

In [36]:
df_p = pd.DataFrame(columns=['x', 'y'])
df_o = pd.DataFrame(columns=['x', 'y'])
df_a = pd.DataFrame(columns=['x', 'y'])

,x,y


In [52]:
row

concept_id                      4000935
concept_name                   BITTER-3
domain_id                          Drug
vocabulary_id                    SNOMED
concept_class_id    Pharma/Biol Product
standard_concept                    NaN
concept_code                  100000000
valid_start_date               20020131
valid_end_date                 20090731
invalid_reason                        D
Name: 6903171, dtype: object

In [56]:
df_concept_rel[df_concept_rel['concept_id_1']==row['concept_id']]

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
1909558,4001100,40642537,Has status,20220128,20991231,NaN
1909559,4001100,40642539,Has Module,20220128,20991231,NaN
2413121,4001100,3397873,Mapped from,19700101,20991231,NaN
2413122,4001100,4001100,Mapped from,19700101,20991231,NaN
2413123,4001100,4001100,Maps to,19700101,20991231,NaN
2413124,4001100,40314079,Concept same_as from,20140401,20991231,NaN
2413125,4001100,40314079,Mapped from,19700101,20991231,NaN
14388908,4001100,4185453,Has dir proc site,20110731,20991231,NaN
21042977,4001100,40368774,Concept same_as from,20140401,20991231,NaN
21042978,4001100,40368774,Mapped from,19700101,20991231,NaN


In [58]:
cnt

40

In [59]:
df_match = snomed_vocabs.merge(df_concept_rel[(df_concept_rel['relationship_id']=='Subsumes')], left_on='concept_id', right_on='concept_id_1', how='left')

In [62]:
(~df_match['concept_id_2'].isna()).sum()

333249

In [60]:
df_match

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date_x,valid_end_date_x,invalid_reason_x,concept_id_1,concept_id_2,relationship_id,valid_start_date_y,valid_end_date_y,invalid_reason_y
0,4001100,Resection of stomach fundus,Procedure,SNOMED,Procedure,S,10002003,20020131,20991231,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4002094,Repair of retinal detachment with xenon arc ph...,Procedure,SNOMED,Procedure,S,10006000,20020131,20991231,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40264452,Retired procedure,Procedure,SNOMED,Procedure,NaN,10009007,19700101,20020131,D,NaN,NaN,NaN,NaN,NaN,NaN
3,4000799,Cauterization of Bartholin's gland,Procedure,SNOMED,Procedure,S,1001000,20020131,20991231,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42534812,Ultrasonography of calf of right lower leg,Procedure,SNOMED,Procedure,S,1001000087107,20180131,20991231,NaN,42534812.0,42538953.0,Subsumes,20180131.0,20991231.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403577,4322524,Radiation therapy treatment planning service,Procedure,SNOMED,Procedure,S,9990009,20020131,20991231,NaN,4322524.0,4044940.0,Subsumes,20110731.0,20991231.0,NaN
403578,4322524,Radiation therapy treatment planning service,Procedure,SNOMED,Procedure,S,9990009,20020131,20991231,NaN,4322524.0,4061543.0,Subsumes,20110731.0,20991231.0,NaN
403579,4322524,Radiation therapy treatment planning service,Procedure,SNOMED,Procedure,S,9990009,20020131,20991231,NaN,4322524.0,4044943.0,Subsumes,20110731.0,20991231.0,NaN
403580,4322532,Incision of diaphragm,Procedure,SNOMED,Procedure,S,9993006,20020131,20991231,NaN,4322532.0,42535710.0,Subsumes,20180131.0,20991231.0,NaN


In [57]:
cnt = 0
for idx, row in snomed_vocabs.iterrows():
    target_ids = df_concept_rel.loc[(df_concept_rel['concept_id_1']==row['concept_id']) & (df_concept_rel['relationship_id']=='Subsumes'), 'concept_id_2'].values
    if len(target_ids) == 0:
        continue
        
    cnt += 1
    continue
    target_ids = np.intersect1d(target_ids, icd10pcs_ids)
    
    codes = pd.DataFrame({'concept_id':target_ids}).merge(df_concept, on='concept_id')['concept_code'].str[:5].values
    asdf
    for code in codes:
        if code[0] == 0:
            # body part
            y_p = code[1]+code[3]
            y_o = code[2]
            y_a = code[4]
            break
    df_p = pd.concat(df_p, pd.DataFrame({'x': row['concept_name'], 'y': y_p}))
    df_o = pd.concat(df_o, pd.DataFrame({'x': row['concept_name'], 'y': y_o}))

        
    target_ids = df_concept_rel.loc[(df_concept_rel['concept_id_1']==row['concept_id']) & (df_concept_rel['relationship_id']=='Has direct site'), 'concept_id_2'].values
    adf
    target_ids = df_concept_rel.loc[(df_concept_rel['concept_id_1']==row['concept_id']) & (df_concept_rel['relationship_id']=='Has Method'), 'concept_id_2'].values
        

KeyboardInterrupt: 

In [46]:
codes

array([], dtype=object)

In [ ]:
df_p

In [ ]:
target_ids

In [43]:
df_concept_rel[df_concept_rel['relationship_id']=='Has method']

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
25024,36203120,1029151,Has method,20191213,20991231,NaN
25036,36203122,1029151,Has method,20191213,20991231,NaN
25045,36203134,1029151,Has method,20191213,20991231,NaN
25058,36203150,1029151,Has method,20191213,20991231,NaN
25066,36203166,1029151,Has method,20191213,20991231,NaN
...,...,...,...,...,...,...
60000766,35608044,4043846,Has method,20180731,20991231,NaN
60001271,35610430,4044525,Has method,20180731,20991231,NaN
60003892,35622740,4044190,Has method,20180731,20991231,NaN
60004300,35624545,4044525,Has method,20180731,20991231,NaN


In [22]:
# hysterectomy (concept ID 4127886, concept code 236886002)
#df_concept_rel.loc[(df_concept_rel['concept_id_1']==4127886) & (df_concept_rel['relationship_id']=='Subsumes') & (df_concept_rel['concept_id_2'].isin(icd10pcs_codes)), 'concpet_id_2']

target_ids = df_concept_rel.loc[(df_concept_rel['concept_id_1']==4127886) & (df_concept_rel['relationship_id']=='Subsumes'), 'concept_id_2'].values
target_ids = np.intersect1d(target_ids, icd10pcs_ids)

NameError: name 'intersect1d' is not defined

In [27]:
df_concept

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,45756805,Pediatric Cardiology,Provider,ABMS,Physician Specialty,S,OMOP4821938,19700101,20991231,NaN
1,45756804,Pediatric Anesthesiology,Provider,ABMS,Physician Specialty,S,OMOP4821939,19700101,20991231,NaN
2,45756803,Pathology-Anatomic / Pathology-Clinical,Provider,ABMS,Physician Specialty,S,OMOP4821940,19700101,20991231,NaN
3,45756802,Pathology - Pediatric,Provider,ABMS,Physician Specialty,S,OMOP4821941,19700101,20991231,NaN
4,45756801,Pathology - Molecular Genetic,Provider,ABMS,Physician Specialty,S,OMOP4821942,19700101,20991231,NaN
...,...,...,...,...,...,...,...,...,...,...
9154804,2213270,Molecular cytogenetics; chromosomal in situ hy...,Measurement,CPT4,CPT4,S,88272,19700101,20991231,NaN
9154805,45889102,"Bone graft, any donor area",Procedure,CPT4,CPT4 Hierarchy,C,1003752,20141010,20991231,NaN
9154806,2213271,Molecular cytogenetics; chromosomal in situ hy...,Measurement,CPT4,CPT4,S,88273,19700101,20991231,NaN
9154807,2213272,Molecular cytogenetics; interphase in situ hyb...,Measurement,CPT4,CPT4,S,88274,19700101,20991231,NaN


In [29]:
codes = pd.DataFrame({'concept_id':target_ids}).merge(df_concept, on='concept_id')['concept_code'].str[:5].values
codes

array(['0UB90', '0UB90', '0UB93', '0UB93', '0UB97', '0UB97', '0UB98',
       '0UB98', '10T23', '10T27'], dtype=object)

### eda

In [82]:
df

,Procedure Code Category,code,name,Code Status,src,0,2,opid,p,o,a,opname,icd_version
0,AAA,04B00ZZ,"Excision of Abdominal Aorta, Open Approach",No change,cdc,NaN,NaN,NaN,40,BB,00,NaN,NaN
1,AAA,04B04ZZ,"Excision of Abdominal Aorta, Percutaneous Endo...",No change,cdc,NaN,NaN,NaN,40,BB,44,NaN,NaN
2,AAA,04R007Z,Replacement of Abdominal Aorta with Autologous...,No change,cdc,NaN,NaN,NaN,40,RR,00,NaN,NaN
3,AAA,04R00JZ,Replacement of Abdominal Aorta with Synthetic ...,No change,cdc,NaN,NaN,NaN,40,RR,00,NaN,NaN
4,AAA,04R00KZ,Replacement of Abdominal Aorta with Nonautolog...,No change,cdc,NaN,NaN,NaN,40,RR,00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85252,NaN,None,Introduction of Gilteritinib Antineoplastic in...,NaN,mimic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
85253,NaN,None,"New Technology, Physiological Systems, Measure...",NaN,mimic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
85254,NaN,None,"Measurement of Infection, Whole Blood Nucleic ...",NaN,mimic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
85255,NaN,None,"New Technology, Extracorporeal, Introduction",NaN,mimic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0


In [58]:
df4

,code,icd_version,name,src
4,001,10,"Central Nervous System and Cranial Nerves, Bypass",mimic
12,0016070,10,Bypass Cerebral Ventricle to Nasopharynx with ...,mimic
13,0016071,10,Bypass Cerebral Ventricle to Mastoid Sinus wit...,mimic
14,0016072,10,Bypass Cerebral Ventricle to Atrium with Autol...,mimic
15,0016073,10,Bypass Cerebral Ventricle to Blood Vessel with...,mimic
...,...,...,...,...
85252,XW0DXV5,10,Introduction of Gilteritinib Antineoplastic in...,mimic
85253,XXE,10,"New Technology, Physiological Systems, Measure...",mimic
85254,XXE5XM5,10,"Measurement of Infection, Whole Blood Nucleic ...",mimic
85255,XY0,10,"New Technology, Extracorporeal, Introduction",mimic


In [78]:
my_list = df4['code'].str[0].values
element_counts = {}
for item in my_list:
    if item in element_counts:
        element_counts[item] += 1
    else:
        element_counts[item] = 1

print(element_counts)

{'0': 70631, '1': 314, '2': 871, '3': 1482, '4': 421, '5': 54, '6': 57, '7': 101, '8': 79, '9': 91, 'B': 3036, 'C': 502, 'D': 2075, 'F': 1394, 'G': 42, 'H': 66, 'X': 153}


In [72]:
df3

,opid,p,o,a,opname,name,src
0,190617045,NaN,NaN,4,Salpingo-oophorectomy ( bilateral / multiport ...,Salpingo-oophorectomy ( bilateral / multiport ...,snuh
1,200715065,NaN,NaN,4,Salpingo-oophorectomy ( left / single port tot...,Salpingo-oophorectomy ( left / single port tot...,snuh
2,190306113,NaN,NaN,4,Ovarian cystectomy ( bilateral / single port t...,Ovarian cystectomy ( bilateral / single port t...,snuh
3,181022121,NaN,NaN,4,Ovarian cystectomy ( right / multiport total l...,Ovarian cystectomy ( right / multiport total l...,snuh
4,201120073,NaN,NaN,NaN,Lobectomy ( LUL / thoracotomy / with mediastin...,Lobectomy ( LUL / thoracotomy / with mediastin...,snuh
...,...,...,...,...,...,...,...
286210,110902018,YY,6,0,"Amputation, 5th toe, Lt.","Amputation, 5th toe, Lt.",snuh
286211,110701008,YY,6,0,"Amputation of toe, foot, 5th , Lt. prn>4th toe...","Amputation of toe, foot, 5th , Lt. prn>4th toe...",snuh
286212,121207059,YY,6,0,"Amputation of toe, 5th toe, Lt.","Amputation of toe, 5th toe, Lt.",snuh
286213,110527015,YY,6,0,Amputation of toe 5th toe. Lt.,Amputation of toe 5th toe. Lt.,snuh


In [67]:
len(df1['code'].unique()), len(df2['code'].unique()), len(df3['opname'].unique()), len(df4['code'].unique())

(8993, 79122, 108294, 81369)

In [17]:
df2

,0,code,2,name,src
0,00001,001,0,"Central Nervous System and Cranial Nerves, Bypass",cms
1,00002,0016070,1,"Bypass Cereb Vent to Nasophar with Autol Sub, ...",cms
2,00003,0016071,1,Bypass Cereb Vent to Mastoid Sinus w Autol Sub...,cms
3,00004,0016072,1,"Bypass Cereb Vent to Atrium with Autol Sub, Op...",cms
4,00005,0016073,1,"Bypass Cereb Vent to Blood Vess w Autol Sub, Open",cms
...,...,...,...,...,...
79117,79118,XXE97U7,1,"Measurement of Nasophrn Fluid SARS-CoV-2 PCR, ...",cms
79118,79119,XXEBXQ6,1,"Measure of Infect, Lwr Resp Fld Microb Detect,...",cms
79119,79120,XY0,0,"New Technology, Extracorporeal, Introduction",cms
79120,79121,XY0VX83,1,Introduce Endoth Dmg Inhib Vn Grft New Tech 3,cms


In [56]:
df1

,Procedure Code Category,code,name,Code Status,src
0,AAA,04B00ZZ,"Excision of Abdominal Aorta, Open Approach",No change,cdc
1,AAA,04B04ZZ,"Excision of Abdominal Aorta, Percutaneous Endo...",No change,cdc
2,AAA,04R007Z,Replacement of Abdominal Aorta with Autologous...,No change,cdc
3,AAA,04R00JZ,Replacement of Abdominal Aorta with Synthetic ...,No change,cdc
4,AAA,04R00KZ,Replacement of Abdominal Aorta with Nonautolog...,No change,cdc
...,...,...,...,...,...
8988,XLAP,0WWJ40Z,"Revision of Drainage Device in Pelvic Cavity, ...",No change,cdc
8989,XLAP,0WWJ41Z,Revision of Radioactive Element in Pelvic Cavi...,No change,cdc
8990,XLAP,0WWJ43Z,"Revision of Infusion Device in Pelvic Cavity, ...",No change,cdc
8991,XLAP,0WWJ4JZ,Revision of Synthetic Substitute in Pelvic Cav...,No change,cdc


In [70]:
df4[df4['code']=='0Y6Y0Z2']

,code,icd_version,name,src
70041,0Y6Y0Z2,10,"Detachment at Left 5th Toe, Mid, Open Approach",mimic


In [71]:
df2[df2['code']=='0Y6Y0Z2']

,0,code,2,name,src
67397,67398,0Y6Y0Z2,1,"Detachment at Left 5th Toe, Mid, Open Approach",cms


In [ ]:
body part -> 1,3
surgery type --> 2
approach --> 4


## modeling

In [6]:
!pip install transformers

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

/bin/bash: /opt/anaconda-3-2020.02/envs/hskim/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 7.7 MB 608 kB/s             
     |████████████████████████████████| 3.8 MB 15.9 MB/s            
     |████████████████████████████████| 1.3 MB 16.9 MB/s            
     |████████████████████████████████| 301 kB 17.2 MB/s            
     |████████████████████████████████| 166 kB 15.7 MB/s            
     |████████████████████████████████| 295 kB 16.4 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
input_path = 'icd10pcs_mapping/Tokenizer'

# shuffling
df = df.sample(frac=1).reset_index(drop=True)

x = np.copy(df['name'].values)

# process x
vocab_size = 4000
t_x = Tokenizer(vocab_size)
t_x.fit_on_texts(x)
x = t_x.texts_to_sequences(x)
maxlen = max(len(e) for e in x)  # fit for max len
x = keras.preprocessing.sequence.pad_sequences(x, maxlen=maxlen)
open(f'{input_path}/tokenizer_x.json', "wt").write(t_x.to_json())

prefix = f'{input_path}/transformer_res'
for target in ('p', 'o', 'a'):
    opath = f'{prefix}_{target}.csv'
    if os.path.exists(opath):
        continue

    y = np.copy(df[target].values) # body part is consistent within a specific body system

    t_y = Tokenizer()
    valid_mask = pd.notnull(y)
    t_y.fit_on_texts(y[valid_mask])
    y[valid_mask] = np.array(t_y.texts_to_sequences(y[valid_mask])).flatten() - 1
    open(f'{input_path}/tokenizer_y_{target}.json', "wt").write(t_y.to_json())

    # train, val, test set
    valid_mask = df[target].notnull()
    tune_mask = valid_mask & (df['src'] == 'snuh')  # non-snuh -> snuh 
    pre_mask = valid_mask & ~tune_mask

    # model structure
    embed_dim = 64  # Embedding size for each token
    num_heads = 2  # Number of attention heads
    ff_dim = 32  # Hidden layer size in feed forward network inside transformer
    droprate = 0.2
    inp = layers.Input(shape=(maxlen,))
    out = inp
    out = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)(out)
    out = TransformerBlock(embed_dim, num_heads, ff_dim, droprate)(out)
    out = layers.GlobalAveragePooling1D()(out)
    out = layers.Dropout(droprate)(out)
    out = layers.Dense(256, activation="relu")(out)
    out = layers.Dropout(droprate)(out)
    out = layers.Dense(len(t_y.word_index), activation="softmax")(out)
    model = keras.Model(inputs=inp, outputs=out)

    # # bio clinical bert (pretrained model)
    # from transformers import TFAutoModelForSequenceClassification
    # model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(t_p.word_index), from_pt=True)

    weight_path = f'{input_path}/weights_{target}.h5'
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    # pre-trian
    print(f'pretraining {target}...')
    y_train_pre = y[pre_mask].astype(int)
    model.fit(x[pre_mask], y_train_pre,
            batch_size=BATCH_SIZE, epochs=NUM_EPOCH, validation_split=0.1,
            callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weight_path, verbose=1, save_best_only=True),
                        EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='auto')])
    model.load_weights(weight_path)  # reload the best model

    # fine tunning
    print(f'fine tuning {target}...')
    weight_path = f'{input_path}/tuned_weights_{target}.h5'
    y_train_tune = y[tune_mask].astype(int)
    model.fit(x[tune_mask], y_train_tune,
            batch_size=BATCH_SIZE, epochs=NUM_EPOCH, validation_split=0.1,
            callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weight_path, verbose=1, save_best_only=True),
                        EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='auto')])
    model.load_weights(weight_path)  # reload the best model
    open(f'{input_path}/model_{target}.json', "wt").write(model.to_json())

    pred = model.predict(x)
    df['pred'] = pd.Series(t_y.sequences_to_texts(np.argmax(pred, axis=1)[...,None] + 1)).str.upper()
    df['conf'] = pred.max(axis=1)
    df['matched'] = (df['pred'] == df[target]).astype(int)

    # save the results for snuh
    df[df['src'] == 'snuh'].drop(columns='src').sort_values(by='opid').to_csv(opath, index=False, encoding='utf-8-sig')
    
    
    
    

# merge
df_a = pd.read_csv(f'{prefix}_a.csv', dtype=str, usecols=['opid', 'pred', 'matched', 'conf'])
df_o = pd.read_csv(f'{prefix}_o.csv', dtype=str, usecols=['opid', 'pred', 'matched', 'conf'])
df = pd.read_csv(f'{prefix}_p.csv', dtype=str)
df = df.merge(df_a, how='left', on='opid', suffixes=('', '_a'))
df = df.merge(df_o, how='left', on='opid', suffixes=('', '_o'))
df.rename(columns={'pred':'pred_p', 'matched':'matched_p'}, inplace=True)
df['code_pred'] = '0'
df.loc[df['name'].str.contains('cesarian'), 'code_pred'] = '1'
df['code_pred'] += df['pred_p'].str[0] + df['pred_o'] + df['pred_p'].str[1] + df['pred_a']
df.to_csv(f'{prefix}.csv', index=False, encoding='utf-8-sig')

FileNotFoundError: [Errno 2] No such file or directory: 'icd10pcs_mapping/Tokenizer/tokenizer_x.json'

### mover conversion

In [5]:
input_path = 'icd10pcs_mapping/Tokenizer'

# shuffling
df = df.sample(frac=1).reset_index(drop=True)
#df_info = pd.read_csv('mover/EPIC_EMR/patient_information.csv')
#df_info['procedure_nm'] = df_info['PRIMARY_PROCEDURE_NM'].apply(lambda x: x if isinstance(x, str) else '')
df_info = pd.read_csv('mover/SIS_EMR/patient_information.csv')
df_info['procedure_nm'] = df_info['Procedure'].apply(lambda x: x if isinstance(x, str) else '')

x = np.copy(df['name'].values)

# process x
vocab_size = 4000
t_x = Tokenizer(vocab_size)
t_x.fit_on_texts(x)
x = t_x.texts_to_sequences(x)
maxlen = max(len(e) for e in x)  # fit for max len
x = keras.preprocessing.sequence.pad_sequences(x, maxlen=maxlen)
open(f'{input_path}/tokenizer_x.json', "wt").write(t_x.to_json())

mover_text = np.copy(df_info['procedure_nm'].values)
mover_x = t_x.texts_to_sequences(mover_text)
mover_x = keras.preprocessing.sequence.pad_sequences(mover_x, maxlen=maxlen)

prefix = f'{input_path}/transformer_res'
for target in ('p', 'o', 'a'):
    opath = f'{prefix}_{target}.csv'
    #if os.path.exists(opath):
    #    continue

    y = np.copy(df[target].values) # body part is consistent within a specific body system

    t_y = Tokenizer()
    valid_mask = pd.notnull(y)
    t_y.fit_on_texts(y[valid_mask])
    y[valid_mask] = np.array(t_y.texts_to_sequences(y[valid_mask])).flatten() - 1
    open(f'{input_path}/tokenizer_y_{target}.json', "wt").write(t_y.to_json())

    # train, val, test set
    valid_mask = df[target].notnull()
    tune_mask = valid_mask & (df['src'] == 'snuh')  # non-snuh -> snuh 
    pre_mask = valid_mask & ~tune_mask

    # model structure
    embed_dim = 64  # Embedding size for each token
    num_heads = 2  # Number of attention heads
    ff_dim = 32  # Hidden layer size in feed forward network inside transformer
    droprate = 0.2
    inp = layers.Input(shape=(maxlen,))
    out = inp
    out = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)(out)
    out = TransformerBlock(embed_dim, num_heads, ff_dim, droprate)(out)
    out = layers.GlobalAveragePooling1D()(out)
    out = layers.Dropout(droprate)(out)
    out = layers.Dense(256, activation="relu")(out)
    out = layers.Dropout(droprate)(out)
    out = layers.Dense(len(t_y.word_index), activation="softmax")(out)
    model = keras.Model(inputs=inp, outputs=out)

    # # bio clinical bert (pretrained model)
    # from transformers import TFAutoModelForSequenceClassification
    # model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(t_p.word_index), from_pt=True)

    weight_path = f'{input_path}/weights_{target}.h5'
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    # pre-trian
    print(f'pretraining {target}...')
    y_train_pre = y[pre_mask].astype(int)
    model.fit(x[pre_mask], y_train_pre,
            batch_size=BATCH_SIZE, epochs=NUM_EPOCH, validation_split=0.1,
            callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weight_path, verbose=1, save_best_only=True),
                        EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='auto')])
    model.load_weights(weight_path)  # reload the best model

    # fine tunning
    print(f'fine tuning {target}...')
    weight_path = f'{input_path}/tuned_weights_{target}.h5'
    y_train_tune = y[tune_mask].astype(int)
    model.fit(x[tune_mask], y_train_tune,
            batch_size=BATCH_SIZE, epochs=NUM_EPOCH, validation_split=0.1,
            callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weight_path, verbose=1, save_best_only=True),
                        EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='auto')])
    model.load_weights(weight_path)  # reload the best model
    open(f'{input_path}/model_{target}.json', "wt").write(model.to_json())

    pred = model.predict(x)
    df['pred'] = pd.Series(t_y.sequences_to_texts(np.argmax(pred, axis=1)[...,None] + 1)).str.upper()
    df['conf'] = pred.max(axis=1)
    df['matched'] = (df['pred'] == df[target]).astype(int)

    # save the results for snuh
    df[df['src'] == 'snuh'].drop(columns='src').sort_values(by='opid').to_csv(opath, index=False, encoding='utf-8-sig')
    
    # mover
    m_pred = model.predict(mover_x)
    df_info[f'pred_{target}'] = pd.Series(t_y.sequences_to_texts(np.argmax(m_pred, axis=1)[...,None] + 1)).str.upper()
    df_info['conf'] = m_pred.max(axis=1)

    # save the results for snuh
    df_info.to_csv(f'{prefix}_{target}_mover_sis.csv', index=False, encoding='utf-8-sig')
    

df_info['code_pred'] = '0'
df_info.loc[df_info['procedure_nm'].str.contains('cesarian'), 'code_pred'] = '1'
df_info['code_pred'] += df_info['pred_p'].str[0] + df_info['pred_o'] + df_info['pred_p'].str[1] + df_info['pred_a']

df_info.to_csv(f'mover/sis_procedure_mapped.csv', index=False, encoding='utf-8-sig')

2023-11-17 15:08:59.783177: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22321 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:c2:00.0, compute capability: 8.6


pretraining p...
Epoch 1/100


2023-11-17 15:09:02.981941: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-17 15:09:03.207307: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x281f7370 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-17 15:09:03.207368: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2023-11-17 15:09:03.327215: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-17 15:09:03.558827: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2023-11-17 15:09:03.713113: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-17 15:09:03.818741: I ./tens

516/516 [==============================] - ETA: 0s - loss: 4.0406 - accuracy: 0.2354
Epoch 1: val_loss improved from inf to 0.49146, saving model to icd10pcs_mapping/Tokenizer/weights_p.h5
516/516 [==============================] - 20s 28ms/step - loss: 4.0406 - accuracy: 0.2354 - val_loss: 0.4915 - val_accuracy: 0.8728
Epoch 2/100
 14/516 [..............................] - ETA: 4s - loss: 0.8319 - accuracy: 0.7720

/home/hskim1/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


512/516 [============================>.] - ETA: 0s - loss: 0.3621 - accuracy: 0.8942
Epoch 2: val_loss improved from 0.49146 to 0.04978, saving model to icd10pcs_mapping/Tokenizer/weights_p.h5
516/516 [==============================] - 5s 11ms/step - loss: 0.3606 - accuracy: 0.8946 - val_loss: 0.0498 - val_accuracy: 0.9815
Epoch 3/100
516/516 [==============================] - ETA: 0s - loss: 0.1232 - accuracy: 0.9601
Epoch 3: val_loss improved from 0.04978 to 0.01618, saving model to icd10pcs_mapping/Tokenizer/weights_p.h5
516/516 [==============================] - 5s 10ms/step - loss: 0.1232 - accuracy: 0.9601 - val_loss: 0.0162 - val_accuracy: 0.9950
Epoch 4/100
511/516 [============================>.] - ETA: 0s - loss: 0.0774 - accuracy: 0.9749
Epoch 4: val_loss improved from 0.01618 to 0.01138, saving model to icd10pcs_mapping/Tokenizer/weights_p.h5
516/516 [==============================] - 5s 9ms/step - loss: 0.0774 - accuracy: 0.9749 - val_loss: 0.0114 - val_accuracy: 0.9969
Ep

/home/hskim1/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


515/516 [============================>.] - ETA: 0s - loss: 0.0014 - accuracy: 0.9999
Epoch 2: val_loss improved from 0.00131 to 0.00007, saving model to icd10pcs_mapping/Tokenizer/weights_o.h5
516/516 [==============================] - 5s 10ms/step - loss: 0.0014 - accuracy: 0.9999 - val_loss: 7.4080e-05 - val_accuracy: 1.0000
Epoch 3/100
516/516 [==============================] - ETA: 0s - loss: 4.8997e-04 - accuracy: 1.0000
Epoch 3: val_loss improved from 0.00007 to 0.00007, saving model to icd10pcs_mapping/Tokenizer/weights_o.h5
516/516 [==============================] - 5s 9ms/step - loss: 4.8997e-04 - accuracy: 1.0000 - val_loss: 7.3679e-05 - val_accuracy: 0.9999
Epoch 4/100
516/516 [==============================] - ETA: 0s - loss: 2.5765e-04 - accuracy: 1.0000
Epoch 4: val_loss improved from 0.00007 to 0.00002, saving model to icd10pcs_mapping/Tokenizer/weights_o.h5
516/516 [==============================] - 5s 9ms/step - loss: 2.5765e-04 - accuracy: 1.0000 - val_loss: 1.8357e-0

/home/hskim1/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


513/516 [============================>.] - ETA: 0s - loss: 3.5841e-04 - accuracy: 0.9999
Epoch 2: val_loss improved from 0.00022 to 0.00001, saving model to icd10pcs_mapping/Tokenizer/weights_a.h5
516/516 [==============================] - 5s 9ms/step - loss: 3.5676e-04 - accuracy: 0.9999 - val_loss: 1.2532e-05 - val_accuracy: 1.0000
Epoch 3/100
516/516 [==============================] - ETA: 0s - loss: 8.3137e-05 - accuracy: 1.0000
Epoch 3: val_loss improved from 0.00001 to 0.00000, saving model to icd10pcs_mapping/Tokenizer/weights_a.h5
516/516 [==============================] - 5s 9ms/step - loss: 8.3137e-05 - accuracy: 1.0000 - val_loss: 3.4716e-06 - val_accuracy: 1.0000
Epoch 4/100
515/516 [============================>.] - ETA: 0s - loss: 2.9432e-04 - accuracy: 0.9999
Epoch 4: val_loss did not improve from 0.00000
516/516 [==============================] - 5s 9ms/step - loss: 2.9496e-04 - accuracy: 0.9999 - val_loss: 0.0015 - val_accuracy: 0.9995
Epoch 5/100
512/516 [============

# Inference

## MOVER

In [13]:
df_info = pd.read_csv('mover/EPIC_EMR/patient_information.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'MOVER/EPIC_EMR/patient_information.csv'

In [13]:
mover_x = np.copy(df_info['PRIMARY_PROCEDURE_NM'].values)
x = t_x.texts_to_sequences(mover_x)

array(['INSERTION, PUBOVAGINAL SLING, WITH CYSTOSCOPY', 'GI EGD DILATION',
       'CRANIECTOMY', ...,
       'FOOT AND ANKLE - MASS OR FOREIGN BODY EXCISION',
       'CYSTOSCOPY, WITH URETERAL STENT INSERTION',
       'INSERTION, PENILE PROSTHESIS'], dtype=object)

In [4]:
from keras.models import model_from_json

part = 'a'
with open(f"{load_path}/model_{part}.json", 'r') as f_json:
    model_json = f_json.read()

model = model_from_json(model_json)
load_weight_path = f'{load_path}/tuned_weights_{part}.h5'
model.load_weights(load_weight_path)

TypeError: 'str' object is not callable

In [2]:
from keras.models import model_from_json

load_path = 'icd10pcs_mapping/Tokenizer'

def load_model(load_path, part):
    # Load a transformer model
    with open(f"{load_path}/model_{part}.json", 'r') as f_json:
        model_json = f_json.read()

    model = model_from_json(model_json)
    load_weight_path = f'{load_path}/tuned_weights_{part}.h5'
    model.load_weights(load_weight_path)
    
    # Load a tokenizer
    with open(f"{load_path}/tokenizer_y_{part}.json", 'r') as f_json:
        tokenizer = f_json.read()

    return model, tokenizer

# Load tokenizer for procedure name
with open(f"{load_path}/tokenizer_x.json", 'r') as f_json:
    t_x1 = f_json.read()

# load model and tokenizer for body part, surgery type, approach
model_a, t_a = load_model(load_path, 'a')
model_o, t_o = load_model(load_path, 'o')
model_p, t_p = load_model(load_path, 'p')

TypeError: 'str' object is not callable